# Version datasets in model training runs

## Introduction

You can version datasets, models, and other file objects as Artifacts in Neptune.

This guide shows how to:
* Keep track of a dataset version in your model training runs with artifacts  
* Query the dataset version from previous runs to make sure you are training on the same dataset version
* Group your Neptune Runs by the dataset version they were trained on

By the end of this guide, you will train a few models making sure that the same dataset was used and see the Runs for this dataset version in the Neptune app. 


[See this example in Neptune](https://app.neptune.ai/o/common/org/data-versioning/experiments?compare=IwdgNMQ&split=tbl&dash=artifacts&viewId=6777136b-938e-4639-943d-3f6bc52f8497)

![image](https://neptune.ai/wp-content/uploads/artifacts-grouped-by-dataset-version.png)

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

* If you are running the notebook on your local machine, you need to have [Python](https://www.python.org/downloads/) and [pip](https://pypi.org/project/pip/) installed.
* If you want to see the example recorded to your own workspace instead:
    * Create a Neptune account → [Take me to registration](https://neptune.ai/register)
    * Create a Neptune project that you will use for tracking metadata → [Tell me more about projects](https://docs.neptune.ai/administration/projects)

## Install Neptune and dependencies

In [ ]:
! pip install neptune-client scikit-learn==0.24.1

## Prepare a model training script

Create a training script where you:
* Specify dataset paths for training and testing
* Define model parameters
* Calculate the score on the test set

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

TRAIN_DATASET_PATH = "../datasets/tables/train.csv"
TEST_DATASET_PATH = "../datasets/tables/test.csv"


def train_model(params, train_path, test_path):
    train = pd.read_csv(train_path)
    test = pd.read_csv(test_path)

    FEATURE_COLUMNS = ["sepal.length", "sepal.width", "petal.length", "petal.width"]
    TARGET_COLUMN = ["variety"]
    X_train, y_train = train[FEATURE_COLUMNS], train[TARGET_COLUMN]
    X_test, y_test = test[FEATURE_COLUMNS], test[TARGET_COLUMN]

    rf = RandomForestClassifier(**params)
    rf.fit(X_train, y_train)

    return rf.score(X_test, y_test)

## Initialize Neptune and create new run

Connect your script to Neptune application and create new run.

In [ ]:
import neptune.new as neptune

run = neptune.init(project="common/data-versioning", api_token="ANONYMOUS")

Click on the link above to open this run in Neptune.

For now, it is empty but keep the tab with the run open to see what happens next.

**Few explanations**

In the above code You tell Neptune: 

* **who you are**: your Neptune API token `api_token` 
* **where you want to send your data**: your Neptune `project`.

At this point you have a new run in Neptune. For now on you will use `run` to log metadata to it.

---

**Note**


Instead of logging data to the public project 'common/quickstarts' as an anonymous user 'neptuner' you can log it to your own project.

To do that:

1. Get your [Neptune API token](https://docs.neptune.ai/getting-started/installation#authentication-neptune-api-token)
2. Pass the token to ``api_token`` argument of ``neptune.init()`` method: ``api_token=YOUR_API_TOKEN``
3. Get your [Neptune project name](https://docs.neptune.ai/getting-started/installation#setting-the-project-name)
3. Pass your project to the `project` argument of the `init()` method.

For example:

```python
neptune.init(project="YOUR_WORKSPACE/YOUR_PROJECT", api_token="YOUR_API_TOKEN")
```

## Add tracking of the dataset version

Save datasets versions as Neptune artifacts

In [ ]:
run["datasets/train"].track_files(TRAIN_DATASET_PATH)
run["datasets/test"].track_files(TEST_DATASET_PATH)

**Note:**

You can also version the entire folder where your datasets are by running

```python
run["datasets"].track_files(DATASET_FOLDER)
```

Also, people often keep track of datasets at the project level with [Project metadata](https://docs.neptune.ai/api-reference/project).

For more information see [Organize and share dataset versions](https://docs.neptune.ai/how-to-guides/data-versioning/organize-and-share-dataset-versions).

## Run model training and log parameters and metrics to Neptune

Log parameters to Neptune

In [ ]:
params = {
    "n_estimators": 5,
    "max_depth": 2,
    "max_features": 1,
}
run["parameters"] = params

Log test score to Neptune

In [ ]:
score = train_model(params, TRAIN_DATASET_PATH, TEST_DATASET_PATH)
run["metrics/test_score"] = score

Get the run ID of your model training from Neptune. 

This will be useful when asserting the same dataset versions on the baseline and new datasets. 

In [ ]:
baseline_run_id = run["sys/id"].fetch()
print(baseline_run_id)

## Stop logging to the current run
<font color=red>**Warning:**</font><br>
Once you are done logging, you should stop tracking the run using the `stop()` method.
This is needed only while logging from a notebook environment. While logging through a script, Neptune automatically stops tracking once the script has completed execution.

In [ ]:
run.stop()

## Add a version check for the training and testing datasets

You can fetch the dataset version hash from the baseline and compare it with the new current version of the dataset.

Create a new Neptune run and track the dataset version:

In [ ]:
new_run = neptune.init(project="common/data-versioning", api_token="ANONYMOUS")

new_run["datasets/train"].track_files(TRAIN_DATASET_PATH)
new_run["datasets/test"].track_files(TEST_DATASET_PATH)

Get the Neptune run object for the baseline model:

In [ ]:
baseline_run = neptune.init(
    project="common/data-versioning",
    api_token="ANONYMOUS",
    run=baseline_run_id,
    mode="read-only",
)

Fetch the dataset version with the `.fetch_hash()` method

In [ ]:
baseline_run["datasets/train"].fetch_hash()

Compare the current dataset version with the baseline dataset version

In [ ]:
new_run.wait()  # force asynchronous logging operations to finish

assert baseline_run["datasets/train"].fetch_hash() == new_run["datasets/train"].fetch_hash()
assert baseline_run["datasets/test"].fetch_hash() == new_run["datasets/test"].fetch_hash()

## Run model training with new parameters

Change the parameters and run model training

In [ ]:
params = {
    "n_estimators": 8,
    "max_depth": 3,
    "max_features": 2,
}
new_run["parameters"] = params

score = train_model(params, TRAIN_DATASET_PATH, TEST_DATASET_PATH)

new_run["metrics/test_score"] = score

## Stop logging to the current run

In [ ]:
new_run.stop()
baseline_run.stop()

## See all model training runs for this dataset version

To see all training runs for a particular dataset version:
* Go to the `Runs table` in the Neptune app
* Click on **+Add column**, type in 'artifacts/train' and click on it to add to the `Runs table`
* Add parameters and test score in the same way
* See that your model training run improved thanks to better parameters because the dataset version didn't change. 

You can also [use ** +Group by**](https://docs.neptune.ai/how-to-guides/neptune-ui/groupby) to group by train dataset versions and find the training runs you care about quickly. 

[See this example in Neptune](https://app.neptune.ai/o/common/org/data-versioning/experiments?compare=IwdgNMQ&split=tbl&dash=artifacts&viewId=6777136b-938e-4639-943d-3f6bc52f8497)

![image](https://neptune.ai/wp-content/uploads/artifacts-grouped-by-dataset-version.png)